In [2]:
import pandas
class TreeNode:
    def __init__(self, name_value, num_occurrences, parent_node):
        self.name = name_value
        self.count = num_occurrences
        self.node_link = None  # link to next node with the same name
        self.parent = parent_node  # needs to be updated
        self.children = {}  # empty dictionary

    def increment(self, num_occurrences):
        self.count += num_occurrences


def update_header(node_to_test, target_node):
    while node_to_test.node_link is not None:  # Traverse to the end of the link
        node_to_test = node_to_test.node_link
    node_to_test.node_link = target_node


def update_tree(items, in_tree, header_table, count):
    if items[0] in in_tree.children:
        in_tree.children[items[0]].increment(count)
    else:
        in_tree.children[items[0]] = TreeNode(items[0], count, in_tree)

        if header_table[items[0]][1] is None:
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_header(header_table[items[0]][1], in_tree.children[items[0]])

    if len(items) > 1:
        update_tree(items[1::], in_tree.children[items[0]], header_table, count)


def create_tree(dataset, min_sup=1):
    header_table = {}
    for transaction in dataset:
        for item in transaction:
            header_table[item] = header_table.get(item, 0) + dataset[transaction]

    for k in list(header_table):
        if header_table[k] < min_sup:
            del(header_table[k])

    freq_item_set = set(header_table.keys())

    if len(freq_item_set) == 0:
        return None, None

    for k in header_table:
        header_table[k] = [header_table[k], None]

    ret_tree = TreeNode('Null Set', 1, None)

    for transaction, count in dataset.items():
        local_d = {}
        for item in transaction:
            if item in freq_item_set:
                local_d[item] = header_table[item][0]
        if len(local_d) > 0:
            ordered_items = [v[0] for v in sorted(local_d.items(), key=lambda p: p[1], reverse=True)]
            update_tree(ordered_items, ret_tree, header_table, count)

    return ret_tree, header_table


def mine_tree(header_table, min_sup, pre_fix, freq_item_list):
    big_l = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1][0])]

    for base_pat in big_l:
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_pat)
        freq_item_list.append(new_freq_set)
        cond_pat_bases = find_prefix_path(base_pat, header_table[base_pat][1])
        my_cond_tree, my_head = create_tree(cond_pat_bases, min_sup)

        if my_head is not None:
            mine_tree(my_head, min_sup, new_freq_set, freq_item_list)


def find_prefix_path(base_pat, tree_node):
    cond_pats = {}
    while tree_node is not None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            cond_pats[frozenset(prefix_path[1:])] = tree_node.count
        tree_node = tree_node.node_link
    return cond_pats


def ascend_tree(leaf_node, prefix_path):
    if leaf_node.parent is not None:
        prefix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, prefix_path)


# Example usage
dataset = {
    frozenset(['milk', 'bread']): 3,
    frozenset(['bread', 'butter', 'jam']): 2,
    frozenset(['milk', 'bread', 'butter', 'jam']): 2,
    frozenset(['milk', 'butter', 'jam']): 1,
}

min_sup = 2
fp_tree, header_table = create_tree(dataset, min_sup)
freq_items = []
mine_tree(header_table, min_sup, set(), freq_items)

print("Frequent Itemsets:", freq_items)


Frequent Itemsets: [{'jam'}, {'jam', 'milk'}, {'jam', 'milk', 'bread'}, {'jam', 'bread'}, {'butter'}, {'milk', 'butter'}, {'milk', 'bread', 'butter'}, {'jam', 'milk', 'bread', 'butter'}, {'jam', 'milk', 'butter'}, {'bread', 'butter'}, {'jam', 'bread', 'butter'}, {'jam', 'butter'}, {'milk'}, {'milk', 'bread'}, {'bread'}]
